3. Implemente el algoritmo de Metrópolis Hastings para hacer el ajuste de un histograma cuyas frecuencias relativas siguen una distribución normal.

In [25]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [26]:
file = 'Gaussiana.csv'
url = 'https://raw.githubusercontent.com/asegura4488/Database/main/MetodosComputacionalesReforma/Gaussiano.csv'

In [27]:
data = pd.read_csv(url,sep=",",engine='python')
X = data.x

In [28]:
transition_model = lambda x: [np.random.normal(x[0],0.5,(1,)),np.random.normal(x[1],0.5,(1,))]

In [29]:
def prior(miu, desvest):
    valor = 0
    if (3 <= miu <=5):
        if (0.5 <= desvest <= 3.5):
            valor = 1
    return valor

In [30]:
def likelihood(miu,desvest,x):
    valor = 1
    for every in range(len(x)):
        valor*= (np.sqrt(2 * np.pi * desvest**2))**(-1)*(np.e**(-(miu-x[every])**2/(2*desvest**2)))
    return valor 

In [31]:
def log_distrb(miu, desvest, x):
    log = np.log(prior(miu, desvest)*likelihood(miu,desvest,x))
    return log

In [32]:
def algoritmo_metropolis(x):
    N = 2*10**4
    miu_ = np.random.uniform(3, 5)
    desvest_ = np.random.uniform(0.5, 3.5)
    log_distrb_ = log_distrb(miu_, desvest_, x)
    
    miu_valores = [miu_]
    desvest_valores = [desvest_]
    
    for every in range(N):
        miu__ = np.random.normal(miu_, 0.1)
        desvest__ = np.random.normal(desvest_, 0.1)
        log_new = log_distrb( miu__, desvest__,x)
        
        if log_new > log_distrb_:
            miu_ = miu__
            desvest_ = desvest__
            log_distrb_ = log_new
            
        else:
            aceptado = np.exp(log_new - log_distrb_)
            
            if np.random.uniform() < aceptado:
                miu_ = miu__
                desvest_ = desvest__
                log_distrb_ = log_new
                
        miu_valores.append(miu_)
        desvest_valores.append(desvest_)
    
    return miu_valores, desvest_valores

In [33]:
miu_selected, desvest_selected = algoritmo_metropolis(X)

mediana_miu=np.percentile(miu_selected,50)
mediana_desvest=np.percentile(desvest_selected,50)

In [34]:
print("La mediana de miu es:", mediana_miu )
print("La mediana de sigma es:", mediana_desvest )

La mediana de miu es: 4.022063728161637
La mediana de sigma es: 1.8245689962127944


In [35]:
miu_low=np.percentile(miu_selected,16)
miu_up=np.percentile(miu_selected,84)
desvest_low=np.percentile(desvest_selected,16)
desvest_up=np.percentile(desvest_selected,84)

In [36]:
print("Los valores sigma+- de mu en un intervalo de confianza del CL = 68% son: ", mediana_miu - miu_low , "y", miu_up - mediana_miu  )
print("Los valores sigma+- de sigma en un intervalo de confianza del CL = 68% son: ", mediana_desvest - desvest_low , "y", -mediana_desvest + desvest_up)

Los valores sigma+- de mu en un intervalo de confianza del CL = 68% son:  0.18127471596429068 y 0.17964893273696614
Los valores sigma+- de sigma en un intervalo de confianza del CL = 68% son:  0.12219247571209269 y 0.13598320658272822


$$
\hat{\mu}=4.01_{0.18}^{0.18} at68\%CL
$$

$$
\hat{\sigma}=1.83_{0.11}^{0.13} at68\%CL
$$